# 2021-04-28 WS4_2 Transfer Learning - Head Model

<mark>To be updated!</mark>

<div class="alert alert-block alert-info">
<b>This tutorial is based on:</b>
<ul><li><a href="https://aqibsaeed.github.io/on-device-activity-recognition#blogpost">On-device Activity Recognition</a> by Aaqib Saeed</li>
<li><a href="https://github.com/tensorflow/examples/blob/master/lite/examples/model_personalization/README.md">TensorFlow Lite Example On-device Model Personalization</a></li>
</ul>
</div>

As you remember, we now use a part of the pre-trained generic model (=base model) and add a head model, which will be trained on a mobile device. 

<img src="https://miro.medium.com/max/1400/1*Bi-_rpBbfhVAz6gi1uIRnw.png" width="800">

A head model is typically a simpler network with a few fully-connected (fc) layers. Base model weights are fixed during conversion, and cannot be changed later. Hence with frozen weights in the main network, we will get to use the pre-trained weights to get important feature activations as bottleneck features into our newly added fully-connected network, and now this new fully-connected network (trained on gathered data on a device) gives us the required inference as per training.

Clone the <a href="https://github.com/osaukh/mobile_computing_lab/tree/master/code/OnDeviceActivityRecognition">repository</a> containing the converter, the Android library and the example app.

Do the following from the converter directory to install <code>tfltransfer</code>:

```bash
# Create a virtualenv. This step is optional but recommended.
virtualenv venv

# Activate the created virtualenv.
source venv/bin/activate

# Install the converter.
pip install -e .
```

After training the network for a fixed number of epochs in a standard way, we will save the encoder part of the network named base (without the last layer) in a TensorFlow SavedModel format. The next step is to define a fine-tuning network named head with additional layers that will be trained on a device while keeping the base model fixed. To keep things simple, we add a fully connected layer with 6 units and an output layer with 2 units (for differentiating between two activities). Here, you can also choose between following optimizers <code>Adam</code> or <code>SGD</code> with <code>optimizers.SGD</code> and <code>optimizers.Adam</code>, respectively. Afterward, we will convert and save this network using the __TFLite Transfer Converter__ that will generate the following five files: bottleneck.tflite, inference.tflite, initialize.tflite, optimizer.tflite, and train_head.tflite in a specified directory.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

model_m = load_model('ws4_tl/saved_model.pbtxt')
model = Model(model_m.input, model_m.get_layer('headlayer').output)
save_model(model, 'ws4_tl/chopped_model.pbtxt', include_optimizer = False,save_format="tf")

# --------------- on-device model conversion ---------------- #

# Model configuration.
num_classes = 2
learning_rate = 0.001
batch_size = 5
l2_rate = 0.0001
hidden_units = 6
input_shape = model.get_layer('headlayer').output.shape


base = bases.SavedModelBase('ws4_tl/chopped_model.pbtxt')
head = tf.keras.Sequential([
    layers.Flatten(input_shape=(1, 80, 8)),
    layers.Dense(
        units=hidden_units,
        activation="relu",
        kernel_regularizer=l2(l2_rate)),
    layers.Dense(
        units=num_classes,
        activation="softmax",
        kernel_regularizer=l2(l2_rate)),
])

# Optimizer is ignored by the converter.
head.compile(loss="categorical_crossentropy", optimizer="adam")

converter = TFLiteTransferConverter(num_classes,
                base,
                heads.KerasModelHead(head),
                optimizers.SGD(learning_rate),
                train_batch_size=batch_size)

converter.convert_and_save('ws4_tl/custom_keras_model')

W0410 15:59:10.930770 4549289408 deprecation.py:506] From /Users/ahinea/work/pyenv/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0410 15:59:11.218513 4549289408 deprecation.py:323] From /Users/ahinea/work/mobile_computing_lab/code/OnDeviceActivityRecognition/converter/tfltransfer/heads/keras_model_head.py:44: export_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
W0410 15:59:11.535582 4549289408 deprecation.py:323] From /Users/ahinea/work/pyenv/lib/python3.7/site-packages/tensorflow_core/python/saved_

W0410 15:59:13.952501 4549289408 meta_graph.py:448] Issue encountered when serializing variables.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
W0410 15:59:13.953075 4549289408 meta_graph.py:448] Issue encountered when serializing trainable_variables.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
/Users/ahinea/work/pyenv/lib/python3.7/site-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)
